# Word2vec Computing
##     This file aimed to take advantage of the pretrained model of FastText data to compute the cosine similarity of the tags extracted from "twreporter" (692 articles, 2211 tags) and "newslens".
##     Besides, the cosine similarity of the tags between each documents would be computed, and the result would be taken into consideration as referring to text similarity.

### Step 1. Load fasttext pretrained word vector model

In [12]:
# !pip install -U gensim
from __future__ import print_function
from gensim.models import KeyedVectors

In [130]:
# Creating the model and save the model
zh_model = KeyedVectors.load_word2vec_format('wiki.zh_classical/wiki.zh_classical.vec')
zh_model.save('zh_model')

In [16]:
# Getting the tokens 
words = []
for word in zh_model.vocab:
    words.append(word)

In [18]:
# Printing out number of tokens available
print("Number of Tokens: {}".format(len(words)))

Number of Tokens: 10696


In [20]:
# Printing out the dimension of a word vector 
print("Dimension of a word vector: {}".format(
    len(zh_model[words[0]])
))

Dimension of a word vector: 300


In [167]:
# Print out the vector of a word 
print("Vector components of a word: {}".format(
    zh_model[words[0]]
))

Vector components of a word: [-1.2137e-01  3.7982e-02 -2.7737e-01  3.2376e-01 -1.4194e-01 -5.6606e-02
 -2.8803e-01  3.5320e-01  1.2195e-01 -1.0090e-01  6.5145e-02  1.3050e-01
  2.4721e-01 -5.2645e-01 -3.5504e-01  1.2690e+00  3.9146e-01 -1.4617e-01
 -1.5835e-02  1.3692e-01  1.6282e-01  2.2253e-01  7.3009e-02 -3.1781e-01
 -2.2186e-01 -2.1988e-01  3.8458e-01  1.1683e-01 -3.5936e-01  6.4307e-01
 -2.1387e-01  4.7144e-01  1.6594e-01 -2.8739e-01 -1.4897e-01  3.5061e-01
 -3.7002e-01  9.1249e-02 -9.1529e-01 -2.4780e-01  1.8264e-01  6.6149e-02
  1.9852e-01 -1.5451e-01  5.0628e-01  6.9605e-03 -2.9538e-01 -2.1830e-01
  1.3802e-01  7.7120e-01 -2.7467e-01 -5.1889e-01  2.6613e-01 -1.9626e-01
  1.5237e+00 -4.6052e-01 -4.1049e-01 -2.7603e-01 -1.6977e-01 -1.8822e-01
  2.8506e-01 -1.3644e-01 -3.1863e-01 -7.0669e-02 -7.0327e-02  1.4909e-01
  7.3487e-04  3.6984e-01 -4.3169e-01  4.4443e-01  2.2603e-01 -7.3055e-01
  4.1653e-01 -1.3083e-01 -5.9000e-03  7.0923e-01  1.1901e-01  5.9910e-01
 -5.1202e-01  2.1702e-

In [226]:
zh_model['微軟']
similarity = zh_model.similarity('你', '微軟')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [247]:
# Pick a word 
find_similar_to = '自由'

In [248]:
# Finding out similar words [default= top 10]
for similar_word in zh_model.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))

Word: 微軟, Similarity: 0.83
Word: 協作, Similarity: 0.82
Word: 書面, Similarity: 0.82
Word: 吾等, Similarity: 0.82
Word: 相關, Similarity: 0.82
Word: 營運, Similarity: 0.82
Word: 俄文, Similarity: 0.81
Word: 計畫, Similarity: 0.81
Word: 協會, Similarity: 0.81
Word: 戶名, Similarity: 0.80


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### Step 2. Access the tags of each documents and get their cosine similarities (twreporter)

In [245]:
import csv
import numpy as np

with open('./tags_seg_twreporter_cl.csv', newline='', encoding="big5",
                 errors='ignore') as csvfile:
    rows_1 = csv.reader(csvfile, delimiter=',')
    
    totoftags = 0
    
    alltags = []
    for row in rows_1:
        row_str = ''.join(row[1])        
        tags_1 = row_str.split(',')
        
        for tag in tags_1:
            alltags.append(tag)
        

with open('./tags_seg_newslens-post_ed.csv', newline='', encoding="big5",
                 errors='ignore') as csvfile:
    rows_2 = csv.reader(csvfile, delimiter=',')
    
    for row in rows_2:        
        row_str = ''.join(row[1])        
        tags_2 = row_str.split(',')
            
        for tag in tags_2:
            alltags.append(tag)
    


In [251]:
len(alltags)

25091

In [195]:
df = {}
for t in tags:
    if t in model.vocab:  ## 確認在訓練資療集當中是否有這個字，沒有這一步會出現錯誤
        df[t] = [for term, score in model.most_similar(t)]  ## 原本會回傳(term, score)的List，現在只抓term
    else:
        print(t, ' not in vocab')
df = pd.DataFrame(df)
df

['中國', ' 人權', ' Tibet', ' 新疆', ' Chinese', ' -', ' English', '  ', ' stories']

In [31]:
#reference
# Step 1. https://blog.manash.me/how-to-use-pre-trained-word-vectors-from-facebooks-fasttext-a71e6d55f27
# Step 2-1. https://blog.gtwang.org/programming/python-csv-file-reading-and-writing-tutorial/